<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/threadLayout_Indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.1 1,024보다 큰 벡터의 합 구하기

1.스레드 레이아웃 결정

- 절차 : 블록 크기 결정 -> 데이터의 크기 및 블록 크기에 따라 그리드 크기 결정
- 블록 크기 결정 시에는 커널의 성능 특성과 GPU 자원의 제한을 고려해야함
- 블록 및 그리드 크기와 레지스터나 공유 메모리 크기 등도 제한이 있는 자원에 속함

2.각 스레드가 접근할 데이터 인덱스 계산
- threadIdx만 사용하서 벡터 내 원소를 접근하는 경우 : 여러 개의 블록을 사용하는 경우 한 블록 내에서는 모두 스레드 번호가 다르지만, 다른 블록에는 번호가 같은 스레드가 존재하여 모든 블록의 n번 스레드는 동일한 원소에 접근하게 됨

=> 우리는 1번 블록의 m번째 스레드는 vector[블록의 크기+m]원소를 처리하도록 해야함

**1차원 블록의 경우 m번째 스레드가 접근할 벡터의 원소**



```
vector[blockDim.x + threadIdx.x]
```

**n번 블록 m번째 스레드가 접근할 벡터의 원소**


```
vector[blockIdx.x*blockDim.x+threadIdx.x]
```


3.계산된 인덱스를 반영한 커널 작성

In [13]:
%%cuda
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <chrono>
#include <iostream>

// The size of the vector
#define NUM_DATA (1024*1024*128)

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int* _a, int* _b, int* _c, int _size) {
	int tID = blockIdx.x*blockDim.x+threadIdx.x;
  if(tID< _size)
	  _c[tID] = _a[tID] + _b[tID]; //마지막 블록의 경우 벡터의 크기를 벗어나는 인덱스를 가져 잘못된 영역에 접근하므로 벡터 크기 이상으로는 작업을 중지하도록 예외처리 필요함
}

int main(void)
{

	int* a, * b, * c, * hc;	// Vectors on the host
	int* da, * db, * dc;	// Vectors on the device

	int memSize = sizeof(int) * NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	hc = new int[NUM_DATA]; memset(hc, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}


	// Vector sum on host (for performance comparision)
  auto hostStart = std::chrono::high_resolution_clock::now();
	for (int i = 0; i < NUM_DATA; i++)
		hc[i] = a[i] + b[i];
  auto hostEnd = std::chrono::high_resolution_clock::now();
  // 밀리초 단위로 경과 시간 계산 (소수점 포함)
  std::chrono::duration<double, std::milli> hostElapsed = hostEnd - hostStart;


	// Memory allocation on the device-side
	cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
	cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
	cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

  cudaEvent_t kernelStart, kernelStop;
  cudaEventCreate(&kernelStart);
  cudaEventCreate(&kernelStop);

  auto GPUstart = std::chrono::high_resolution_clock::now();

	// Data copy : Host -> Device
	auto h2dStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);
	auto h2dEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> h2dElapsed = h2dEnd - h2dStart;

  //thread Layout : host 코드임
  auto threadLayoutStart = std::chrono::high_resolution_clock::now();
  dim3 dimGrid(ceil((float)NUM_DATA/256),1,1);
  dim3 dimBlock(256,1,1);
  auto threadLayoutEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> threadLayoutElapsed = threadLayoutEnd - threadLayoutStart;

	// Kernel call
	cudaEventRecord(kernelStart, 0);
	vecAdd <<<dimGrid, dimBlock >>> (da, db, dc, NUM_DATA);
  cudaEventRecord(kernelStop, 0);
  cudaEventSynchronize(kernelStop);
  float kernelTime;
  cudaEventElapsedTime(&kernelTime, kernelStart, kernelStop);


	// Copy results : Device -> Host
  auto d2hStart = std::chrono::high_resolution_clock::now();
	cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);
  auto d2hEnd = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> d2hElapsed = d2hEnd - d2hStart;

	auto GPUend = std::chrono::high_resolution_clock::now();
  std::chrono::duration<double, std::milli> GPUelapsed = GPUend - GPUstart;

	// Release device memory
	cudaFree(da); cudaFree(db); cudaFree(dc);

  // 결과 출력 (밀리초 단위, 소수점 포함)
  std::cout << "Host time: " << hostElapsed.count() << " ms" << std::endl;
  std::cout<<"Host -> Device: " << h2dElapsed.count() << " ms" << std::endl;
  std::cout<<"Kernel: " << threadLayoutElapsed.count() + kernelTime << " ms" << std::endl;
  std::cout<<"Device -> Host: " << d2hElapsed.count() << " ms" << std::endl;
  std::cout<<"CUDA Total Time: " << GPUelapsed.count() << " ms" << std::endl;

  cudaEventDestroy(kernelStart);
  cudaEventDestroy(kernelStop);

	// Check results
	bool result = true;
	for (int i = 0; i < 1; i++) {
		if (hc[i] != c[i]) {
			printf("[%d] The result is not matched! (%d, %d)\n"
				, i, hc[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release host memory
	delete[] a; delete[] b; delete[] c; delete[] hc;

	return 0;
}

134217728 elements, memSize = 536870912 bytes
Host time: 393.343 ms
Host -> Device: 229.888 ms
Kernel: 6.29047 ms
Device -> Host: 112.496 ms
CUDA Total Time: 348.766 ms
GPU works well!



결과
- CPU연산보다 GPU연산 시 명백히 더 빠른 연산 성능을 보임

주의할 점
- 연산 시간 뿐만 아니라 데이터 전송 시간을 고려해야함
- 데이터 전송 시간 고려시 큰 차이가 나지 않음

=> 필요한 연산 양에 비해 데이터 전송 시간의 부하가 매우 커서 CPU사용 대비 효율이 낮음

=> 단순한 연산의 경우 연산 속도보다 데이터 전송 시간의 부하가 더 커질 수도 있음

- GPU 스레드 레이아웃은 GPU 환경 설정과 관련된 부분으로 CPU에서 동작함